In [1]:
import pandas as pd

# This loads a file from the Texas Election Commission (not included in repo yet)

tx = pd.read_csv("../data/inputs/TEC_CF_CSV/filers.csv")

/Users/Matt/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (9,38,43,50,54,79,80,109,112,131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
tx = tx[tx["filerTypeCd"] == 'COH']

In [3]:
txRep = tx[tx['ctaSeekOfficeCd'].isin(['STATEREP', 'STATESEN'])]

In [4]:
txRep[20:30]

,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,...,chairMailingAddr2,chairMailingCity,chairMailingStateCd,chairMailingCountyCd,chairMailingCountryCd,chairMailingPostalCode,chairMailingRegion,chairPrimaryUsaPhoneFlag,chairPrimaryPhoneNumber,chairPrimaryPhoneExt
1705,FILER,19184,COH,"Brown, Mark A. (Mr.)",N,N,N,NaN,STATEREP,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1737,FILER,19413,COH,The Estate of Dan Kubiak,N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1744,FILER,19422,COH,"Laney, James E. (Mr.)",N,N,NaN,NaN,STATEREP,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1757,FILER,19498,COH,"Straus III, Joe R. (The Honorable)",N,N,NaN,NaN,STATEREP,121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1765,FILER,19534,COH,"Sadler, Paul L. (Mr.)",N,N,NaN,NaN,STATESEN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1772,FILER,19581,COH,"Whitmire, John (The Honorable)",N,N,NaN,NaN,STATESEN,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1774,FILER,19595,COH,"Shapleigh, Eliot G. (Mr.)",N,N,NaN,NaN,STATESEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1786,FILER,19657,COH,"Adams, Kent M. (The Honorable)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1802,FILER,19673,COH,"Allen, Alma A. (The Honorable)",N,N,NaN,NaN,STATEREP,131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1805,FILER,19676,COH,"Allen, Phil L. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
filerName = txRep["filerName"]
filerName[615:625]

6215                    Sell, Jeffrey Z.
6216    Fisher, David J. (The Honorable)
6219       Elliott Jr., Charles P. (Mr.)
6244                 Cole, Mark J. (Mr.)
6257           Cannata, Kenneth S. (Mr.)
6263            Hughston, Chris J. (Mr.)
6268     Korn, Andrew R. (The Honorable)
6272           Host Jr., George A. (Mr.)
6277             Elizondo, John A. (Mr.)
6285           Aguilar, Anthony A. (Mr.)
Name: filerName, dtype: object

In [6]:
import re

# The names I try to parse here looked so standardized at first.

# pattern = "([^,]+), ([^\(\. ]+)( ([^\(\. ]+)\.)? \((.+)\)"
# old = "([^,]+)( Jr\.?|Sr\.?|III)?, ([^\( ]+\.?)(\s+\w+\.?(\w\.)?)?(\s+\w+\.?)?\s+\((.+)\)"

pattern = "([^,]+), ([^\( ]+\.?)(\s+[^\( ]+\.?(\w\.)?)?(\s+[^\( ]\.?)?\s+\((.+)\)"
estate = "The estate of (\w+) (\w+)"
noTitle = "([^,]+)( Jr\.|Sr\.)?, ([^\( ]+\.?)(\s+\w+\.?(\w\.)?)?(\s+\w+\.?)?"

def parsename(name):
    try:
        match = re.search(pattern, name)
        return match.group(1), match.group(2), match.group(3), match.group(6)
    except:
        try:
            match = re.search(estate, name, re.IGNORECASE)
            return match.group(2), match.group(1), None, None
        except:
            match = re.search(noTitle, name)
            return match.group(1), match.group(3), match.group(4), None
    
# filerName = txRep["filerName"]

# a new dataframe to join to txRep. 

newcols = pd.DataFrame(txRep["filerIdent"])

newcols["last_name"], newcols["first_name"], newcols["Middle"], newcols["Title"] = \
    zip(*txRep["filerName"].map(parsename))


In [7]:
pattern = " (Jr\.|Sr\.|III)$"

def parsesuffix(last):
    try:
        match = re.search(pattern, last)
        newLast = last.split(match.group(0))[0]
        return newLast, match.group(1)
    except:
        return last, None

newcols["last_name"], newcols["suffixes"] = zip(*newcols["last_name"].map(parsesuffix))

In [8]:
newcols[:5]

,filerIdent,last_name,first_name,Middle,Title,suffixes
0,10066,Lucero,Homero,R.,Mr.,None
1,10144,Criss,Lloyd,W.,Mr.,None
2,10191,Lee,Randy,M.,Mr.,None
14,12312,Walker,Gary,L.,Mr.,None
18,12833,Hernandez,Francisco,None,The Honorable,Jr.


In [9]:
txRep[:5]

,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,...,chairMailingAddr2,chairMailingCity,chairMailingStateCd,chairMailingCountyCd,chairMailingCountryCd,chairMailingPostalCode,chairMailingRegion,chairPrimaryUsaPhoneFlag,chairPrimaryPhoneNumber,chairPrimaryPhoneExt
0,FILER,10066,COH,"Lucero, Homero R. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FILER,10144,COH,"Criss, Lloyd W. (Mr.)",N,N,NaN,NaN,STATEREP,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FILER,10191,COH,"Lee, Randy M. (Mr.)",N,N,NaN,NaN,STATEREP,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,FILER,12312,COH,"Walker, Gary L. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,FILER,12833,COH,"Hernandez Jr., Francisco (The Honorable)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
txNames = pd.merge(txRep, newcols, how='left', on='filerIdent')

In [11]:
txNames[:5]

,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,...,chairMailingPostalCode,chairMailingRegion,chairPrimaryUsaPhoneFlag,chairPrimaryPhoneNumber,chairPrimaryPhoneExt,last_name,first_name,Middle,Title,suffixes
0,FILER,10066,COH,"Lucero, Homero R. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,Lucero,Homero,R.,Mr.,None
1,FILER,10144,COH,"Criss, Lloyd W. (Mr.)",N,N,NaN,NaN,STATEREP,23,...,NaN,NaN,NaN,NaN,NaN,Criss,Lloyd,W.,Mr.,None
2,FILER,10191,COH,"Lee, Randy M. (Mr.)",N,N,NaN,NaN,STATEREP,52,...,NaN,NaN,NaN,NaN,NaN,Lee,Randy,M.,Mr.,None
3,FILER,12312,COH,"Walker, Gary L. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,Walker,Gary,L.,Mr.,None
4,FILER,12833,COH,"Hernandez Jr., Francisco (The Honorable)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,Hernandez,Francisco,None,The Honorable,Jr.


In [12]:
# This loads a file from an Open States bulk download (not included in repo yet)

openstates = pd.read_csv("../data/inputs/openstates/2017-06-02-tx-csv/tx_legislators.csv")

In [13]:
openstates[:5]

,leg_id,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at
0,TXL000182,Kip Averitt,Kip,NaN,Averitt,NaN,NaN,False,tx,NaN,NaN,NaN,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.033000,2011-01-11 18:41:26.081000
1,TXL000183,John Carona,John,NaN,Carona,NaN,NaN,False,tx,NaN,NaN,NaN,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.035000,2015-01-07 22:02:52.111000
2,TXL000184,Wendy Davis,Wendy,NaN,Davis,NaN,NaN,False,tx,NaN,NaN,NaN,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.049000,2015-01-07 22:02:52.109000
3,TXL000185,Bob Deuell,Bob,NaN,Deuell,NaN,NaN,False,tx,NaN,NaN,NaN,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.051000,2015-01-07 22:02:52.115000
4,TXL000186,David Dewhurst,David,NaN,Dewhurst,NaN,NaN,False,tx,NaN,NaN,Republican,http://upload.wikimedia.org/wikipedia/commons/...,2010-06-19 03:51:42.053000,2016-05-02 08:30:02.125000


In [14]:
openTxNames = pd.merge(txNames, openstates, how='left', on=['first_name', 'last_name'])

In [15]:
len(openstates)

328

In [16]:
len(openTxNames)

2509

In [17]:
openTxNames["leg_id"].describe()

count           183
unique          181
top       TXL000289
freq              2
Name: leg_id, dtype: object

In [18]:
openTxNames[:5]

,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,...,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at
0,FILER,10066,COH,"Lucero, Homero R. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FILER,10144,COH,"Criss, Lloyd W. (Mr.)",N,N,NaN,NaN,STATEREP,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FILER,10191,COH,"Lee, Randy M. (Mr.)",N,N,NaN,NaN,STATEREP,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FILER,12312,COH,"Walker, Gary L. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FILER,12833,COH,"Hernandez Jr., Francisco (The Honorable)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
"""
legislators_tlo.html was copied from the source of
http://www.legis.state.tx.us/Search/BillSearchLegislatorList.aspx?ID=usrLegislatorsFolder$cboAuthor&Leg=85
"""

# from bs4 import BeautifulSoup

pattern = """\t<option value="(A[0-9]{3,5})">([^,]+)(, (.+))? \((H|S)-A[0-9]{3,5}\)</option>"""

fp = open('../data/inputs/legislators_tlo.html', encoding = 'utf-8')
lines = fp.readlines()

leg = []

for line in lines:
    # Regex applied to each line 
    match = re.search(pattern, line)
    legislator = [match.group(1), match.group(2), match.group(4), match.group(5)]
    if legislator[3] == "H":
        legislator[3] = "STATEREP"
    else: 
        legislator[3] = "STATESEN"
    leg.append(legislator)
        
leg[:10]

[['A2100', 'Allen', None, 'STATEREP'],
 ['A2125', 'Alonzo', None, 'STATEREP'],
 ['A2135', 'Alvarado', None, 'STATEREP'],
 ['A2150', 'Anchia', None, 'STATEREP'],
 ['A2155', 'Anderson', 'Charles &quot;Doc&quot;', 'STATEREP'],
 ['A2215', 'Anderson', 'Rodney', 'STATEREP'],
 ['A3555', 'Ar&#233;valo', None, 'STATEREP'],
 ['A2330', 'Ashby', None, 'STATEREP'],
 ['A3200', 'Bailes', None, 'STATEREP'],
 ['A2335', 'Bell', None, 'STATEREP']]

In [20]:
leg = pd.DataFrame(leg, columns = ["TLO_id","last_name","TLO_first_name", "ctaSeekOfficeCd"])

In [21]:
leg[:5]

,TLO_id,last_name,TLO_first_name,ctaSeekOfficeCd
0,A2100,Allen,None,STATEREP
1,A2125,Alonzo,None,STATEREP
2,A2135,Alvarado,None,STATEREP
3,A2150,Anchia,None,STATEREP
4,A2155,Anderson,Charles &quot;Doc&quot;,STATEREP


In [22]:
legTxNames = pd.merge(txNames, leg, how='left', on=['last_name', 'ctaSeekOfficeCd'])

In [23]:
leg["TLO_id"].describe()

count       182
unique      182
top       A1055
freq          1
Name: TLO_id, dtype: object

In [24]:
legTxNames["TLO_id"].describe()

count       396
unique      164
top       A3345
freq         16
Name: TLO_id, dtype: object

In [25]:
legTxNames

,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,...,chairPrimaryUsaPhoneFlag,chairPrimaryPhoneNumber,chairPrimaryPhoneExt,last_name,first_name,Middle,Title,suffixes,TLO_id,TLO_first_name
0,FILER,10066,COH,"Lucero, Homero R. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,Lucero,Homero,R.,Mr.,None,NaN,NaN
1,FILER,10144,COH,"Criss, Lloyd W. (Mr.)",N,N,NaN,NaN,STATEREP,23,...,NaN,NaN,NaN,Criss,Lloyd,W.,Mr.,None,NaN,NaN
2,FILER,10191,COH,"Lee, Randy M. (Mr.)",N,N,NaN,NaN,STATEREP,52,...,NaN,NaN,NaN,Lee,Randy,M.,Mr.,None,NaN,NaN
3,FILER,12312,COH,"Walker, Gary L. (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,Walker,Gary,L.,Mr.,None,NaN,NaN
4,FILER,12833,COH,"Hernandez Jr., Francisco (The Honorable)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,Hernandez,Francisco,None,The Honorable,Jr.,A3155,None
5,FILER,13360,COH,"Hodge, Ann F. (Mrs.)",N,N,NaN,NaN,STATEREP,132,...,NaN,NaN,NaN,Hodge,Ann,F.,Mrs.,None,NaN,NaN
6,FILER,13378,COH,"Carter, Charles A. (Mr.)",N,N,NaN,NaN,STATEREP,20,...,NaN,NaN,NaN,Carter,Charles,A.,Mr.,None,NaN,NaN
7,FILER,13764,COH,"Benavides III, Carlos Y. (Mr.)",N,N,NaN,NaN,STATEREP,42,...,NaN,NaN,NaN,Benavides,Carlos,Y.,Mr.,III,NaN,NaN
8,FILER,13800,COH,"Tounget, Craig (Mr.)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,NaN,Tounget,Craig,None,Mr.,None,NaN,NaN
9,FILER,13805,COH,"Hinojosa, Juan (The Honorable)",N,N,NaN,NaN,STATESEN,20,...,NaN,NaN,NaN,Hinojosa,Juan,None,The Honorable,None,A1250,None


In [26]:
allThree = pd.merge(legTxNames, openstates, how='left', on=['first_name', 'last_name'])

In [27]:
# This tests to see which rows actually got all three indexes

cross = allThree[allThree['filerIdent'].notnull()][allThree['TLO_id'].notnull()][allThree['leg_id'].notnull()]

/Users/Matt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [28]:
cross = cross.set_index(['leg_id'])

In [29]:
cross[:5]

,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,...,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at
leg_id,,,,,,,,,,,,,,,,,,,,,
TXL000347,FILER,19498,COH,"Straus III, Joe R. (The Honorable)",N,N,NaN,NaN,STATEREP,121,...,NaN,NaN,True,tx,lower,121.0,Republican,http://www.house.state.tx.us/photos/members/45...,2010-06-19 03:51:42.604000,2017-06-01 10:35:57.467000
TXL000211,FILER,19581,COH,"Whitmire, John (The Honorable)",N,N,NaN,NaN,STATESEN,15,...,NaN,NaN,True,tx,upper,15.0,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.115000,2017-06-01 10:38:26.774000
TXL000244,FILER,19811,COH,"Davis, Yvonne (The Honorable)",N,N,NaN,NaN,STATEREP,111,...,NaN,NaN,True,tx,lower,111.0,Democratic,http://www.house.state.tx.us/photos/members/26...,2010-06-19 03:51:42.219000,2017-06-01 10:38:26.450000
TXL000244,FILER,19811,COH,"Davis, Yvonne (The Honorable)",N,N,NaN,NaN,STATEREP,111,...,NaN,NaN,True,tx,lower,111.0,Democratic,http://www.house.state.tx.us/photos/members/26...,2010-06-19 03:51:42.219000,2017-06-01 10:38:26.450000
TXL000239,FILER,20051,COH,"Craddick, Tom (The Honorable)",N,N,NaN,NaN,STATEREP,NaN,...,NaN,NaN,True,tx,lower,82.0,Republican,http://www.house.state.tx.us/photos/members/26...,2010-06-19 03:51:42.207000,2017-06-01 10:38:26.697000


In [30]:
cross.to_csv('../data/legislator_crosswalk.csv', columns = ['filerIdent', 'TLO_id', 'first_name', 'last_name', \
                                                 'suffix', 'ctaSeekOfficeCd'])